# 5.1 pandas的数据结构介绍

In [1]:
import pandas as pd
import numpy as np

## 5.1.1 Series

Series是一种类似于一维数组的对象,它由一组数据(各种NumPy数据类型)以及一组与之相关的数据标签(即索引)组成。  
仅由一组数据即可产生最简单的Series:

In [2]:
obj = pd.Series([4, 7, -5, 3])

In [3]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

Series的字符串表现形式为:索引在左边,值在右边。由于我们没有为数据指定索引,于是会自动创建一个0到N-1(N为数据的长度)的整数型索引。  
你可以通过Series 的values和index属性获取其数组表示形式和索引对象:

In [4]:
obj.values

array([ 4,  7, -5,  3])

In [5]:
obj.index # # like range(4)

RangeIndex(start=0, stop=4, step=1)

通常,我们希望所创建的Series带有一个可以对各个数据点进行标记的索引:

In [6]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])

In [7]:
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [8]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

与普通NumPy数组相比,你可以在从数据中选择数据的时候使用标签来进行索引:

In [9]:
obj2['a']

-5

In [10]:
obj2['d'] = 6

In [11]:
obj2

d    6
b    7
a   -5
c    3
dtype: int64

\['c', 'a', 'd'\]是索引列表,即使它包含的是字符串而不是整数。

使用NumPy函数或类似NumPy的运算(如根据布尔型数组进行过滤、标量乘法、应用数学函数等)都会保留索引值的链接:

In [12]:
obj2[obj2 > 0]

d    6
b    7
c    3
dtype: int64

In [13]:
obj2 * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [14]:
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

还可以将Series看成是一个定长的有序字典,因为它是索引值到数据值的一个映射。它可以用在许多原本需要字典参数的函数中:

In [15]:
'b' in obj2

True

In [16]:
'e' in obj2

False

如果数据被存放在一个Python字典中,也可以直接通过这个字典来创建Series:

In [17]:
sdata = {'Ohio': 35000, 'Texas': 71000, 'Oregon': 16000, 'Utah': 5000 }

In [18]:
obj3 = pd.Series(sdata)

In [19]:
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [20]:
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = pd.Series(sdata, index=states)

In [21]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

在这个例子中,sdata中跟states索引相匹配的那3个值会被找出来并放到相应的位置上,但由于"California"所对应的sdata值找不到,所以其结果就为NaN(即“非数字”(not a number),在pandas中,它用于表示缺失或NA值)。  
因为‘Utah’不在states中,它被从结果中除去。

我将使用缺失(missing)或NA表示缺失数据。pandas的isnull和notnull函数可用于检测缺失数据:

In [22]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [23]:
pd.notnull(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

Series也有类似的实例方法:

In [24]:
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

对于许多应用而言,Series最重要的一个功能是,它会根据运算的索引标签自动对齐数据:

In [25]:
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [26]:
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

In [27]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

Series对象本身及其索引都有一个name属性,该属性跟pandas其他的关键功能关系非常密切:

In [28]:
obj4.name = 'population'

In [29]:
obj4.index.name = 'state'

In [30]:
obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

Series的索引可以通过赋值的方式就地修改:

In [31]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [32]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']

In [33]:
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

In [34]:
obj.index

Index(['Bob', 'Steve', 'Jeff', 'Ryan'], dtype='object')

## 5.1.2 DataFrame

DataFrame是一个表格型的数据结构,它含有一组有序的列,每列可以是不同的值类型(数值、字符串、布尔值等)。  
DataFrame既有行索引也有列索引,它可以被看做由Series组成的字典(共用同一个索引)。  
DataFrame中的数据是以一个或多个二维块存放的(而不是列表、字典或别的一维数据结构)。  
有关DataFrame内部的技术细节远远超出了本书所讨论的范围。

建DataFrame的办法有很多,最常用的一种是直接传入一个由等长列表或NumPy数组组成的字典:

In [35]:
data = {'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year': [2000, 2001, 2002, 2001, 2002, 2003],
        'pop': [1.5, 1.7, 3.6, 2.4, 2.9, 3.2]}
frame = pd.DataFrame(data)

结果DataFrame会自动加上索引(跟Series一样),且全部列会被有序排列:

In [36]:
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


对于特别大的DataFrame,head方法会选取前五行:

In [37]:
frame.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


如果指定了列序列,则DataFrame的列就会按照指定顺序进行排列:

In [38]:
pd.DataFrame(data, columns=['year', 'pop', 'state'])

,year,pop,state
0,2000,1.5,Ohio
1,2001,1.7,Ohio
2,2002,3.6,Ohio
3,2001,2.4,Nevada
4,2002,2.9,Nevada
5,2003,3.2,Nevada


如果传入的列在数据中找不到,就会在结果中产生缺失值:

In [39]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                           index=['one', 'two', 'three', 'four', 'five', 'six'])

In [40]:
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


In [41]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

通过类似字典标记的方式或属性的方式,可以将DataFrame的列获取为一个Series:

In [42]:
frame2['state']

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object

In [43]:
frame2.year

one      2000
two      2001
three    2002
four     2001
five     2002
six      2003
Name: year, dtype: int64

### <font color="#B22222">注意1</font>
<font color="#1E90FF">
    <strong>笔记:<br>
        IPython提供了类似属性的访问(即frame2.year)和tab补全。<br>
        frame2[column]适用于任何列的名,但是frame2.column只有在列名是一个合理的Python变量名时才适用。
    </strong>
</font>

注意,返回的Series拥有原DataFrame相同的索引,且其name属性也已经被相应地设置好了。

行也可以通过位置或名称的方式进行获取,比如用loc属性(稍后将对此进行详细讲解):

In [44]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

列可以通过赋值的方式进行修改。例如,我们可以给那个空的"debt"列赋上一个标量值或一组值:

In [45]:
frame2['debt'] = 16.5

In [46]:
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5
six,2003,Nevada,3.2,16.5


In [47]:
frame2['debt'] = np.arange(6.)

In [48]:
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0
six,2003,Nevada,3.2,5.0


将列表或数组赋值给某个列时,其长度必须跟DataFrame的长度相匹配。  
如果赋值的是一个Series,就会精确匹配DataFrame的索引,并且在空缺的地方填充缺失值:

In [49]:
val = pd.Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
frame2['debt'] = val

In [50]:
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7
six,2003,Nevada,3.2,NaN


为不存在的列赋值会创建出一个新列。关键字del用于删除列。

作为del的例子,我先添加一个新的布尔值的列,state是否为'Ohio':

In [51]:
frame2['eastern'] = frame2.state == 'Ohio'

In [52]:
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False
six,2003,Nevada,3.2,NaN,False


### <font color="#B22222">注意2</font>
<font color="#1E90FF">
    <strong> 
        注意:<br>
        不能用frame2.eastern创建新的列。
    </strong>
</font>

del方法可以用来删除这列:

In [53]:
del frame2['eastern']

In [54]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

### <font color="#B22222">注意3</font>
<font color="#1E90FF">
    <strong>
        注意:<br>
        通过索引方式返回的列只是相应数据的视图而已,并不是副本。<br>
        因此,对返回的Series所做的任何就地修改全都会反映到源DataFrame上。<br>
        通过Series的copy方法即可指定复制列。
    </strong>
</font>

另一种常见的数据形式是嵌套字典:

In [55]:
pop = {'Nevada': {2001: 2.4, 2002: 2.9},
       'Ohio': {2000: 1.5, 2001:1.7, 2002: 3.6}}

如果嵌套字典传给DataFrame,pandas就会被解释为:
<strong>
    <font color="#CD8500">
        外层
    </font>
    字典的键作为
    <font color="#CD8500">
        列
    </font>
    ,
    <font color="#CD2626">
        内层
    </font>
    键则作为
    <font color="#CD2626">
        行
    </font>
    索引:
</strong>

In [56]:
frame3 = pd.DataFrame(pop)

In [57]:
frame3

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


你也可以使用类似NumPy数组的方法,对DataFrame进行转置(交换行和列):

In [58]:
frame3.T

,2000,2001,2002
Nevada,NaN,2.4,2.9
Ohio,1.5,1.7,3.6


内部字典的键被联合、排序后形成了结果的索引。如果已经显式指明索引的话，内部字典的键将不会被排序：

In [270]:
# pd.DataFrame(pop, columns=['Nevada', 'Ohio'], index=[2001, 2002, 2003])
test_index = pd.Index([2001, 2002, 2003])
pd.DataFrame(pop, index=test_index)

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


### <font color="#B22222">未知错误1</font>

```python
pd.DataFrame(pop, index=[2001, 2002, 2003])
```
<font color="#DC143C">
    <strong>
        上面这行代码会报错，暂时不知到什么原因
    </strong>
</font>

包含Series的字典也可以用于构造DataFrame:

In [60]:
pdata = {'Ohio': frame3['Ohio'][:-1], 'Nevada': frame3['Nevada'][:2]}

In [61]:
pd.DataFrame(pdata)

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4


如果DataFrame的索引和列拥有name属性,则这些name属性也会被显示:

In [267]:
frame3.index.name = 'year'
frame3.columns.name = 'state'

In [268]:
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


和Series类似,DataFrame的values属性会将包含在DataFrame中的数据以二维ndarray的形式返回:

In [64]:
frame3.values

array([[nan, 1.5],
       [2.4, 1.7],
       [2.9, 3.6]])

如果DataFrame的列是不同的dtypes,则values的dtype会自动选择适合所有列的类型:

In [65]:
frame2.values

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, -1.2],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, -1.5],
       [2002, 'Nevada', 2.9, -1.7],
       [2003, 'Nevada', 3.2, nan]], dtype=object)

## 5.1.3 索引对象

In [273]:
obj = pd.Series(range(3), index=['a', 'b', 'c'])

In [274]:
index = obj.index
obj.values

array([0, 1, 2])

In [68]:
index

Index(['a', 'b', 'c'], dtype='object')

In [69]:
index[1:]

Index(['b', 'c'], dtype='object')

### <font color="#B22222">重点1</font>

<font color="#CD2626">
    <strong>
        索引对象是不可变的,因此用户是无法修改索引对象的:
    </strong>
</font>

In [70]:
import traceback

try:
    index[1] = 'd'
except Exception:
    traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-70-7a1658dc02c3>", line 4, in <module>
    index[1] = 'd'
  File "/home/y/anaconda2/envs/notebook/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2065, in __setitem__
    raise TypeError("Index does not support mutable operations")
TypeError: Index does not support mutable operations


不变性使得在多种数据结构中分享索引对象更为安全:

In [71]:
labels = pd.Index(np.arange(3))

In [72]:
labels

Int64Index([0, 1, 2], dtype='int64')

In [73]:
obj2 = pd.Series([1.5, -2.5, 0], index=labels)

In [74]:
obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

In [75]:
obj2.index is labels

True

除了类似数组,索引对象也像一个固定大小的集合:

In [76]:
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [77]:
frame3.columns

Index(['Nevada', 'Ohio'], dtype='object', name='state')

In [78]:
'Ohio' in frame3.columns

True

In [79]:
'Ohio' in frame3.index

False

与Python集合不同,pandas索引对象可以包含重复标签:

In [80]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])

In [81]:
dup_labels

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')

根据重复标签进行筛选,会选取所有重复标签对应的数据。

# 5.2 基本功能

## 5.2.1 重建索引

reindex是pandas对象的重要方法,该方法用于创建一个符合新索引的新对象。考虑下面的例子:

In [82]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])

In [83]:
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

Series调用reindex方法时,会将数据按照新的索引进行排列,如果某个索引值之前并不存在,则会引入缺失值:

In [84]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])

In [85]:
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

对于顺序数据,比如时间序列,在重建索引时可能会需要进行插值或填值。  
method可选参数允许我们使用诸如ffill等方法在重建索引时插值,ffill方法会将值前向填充

In [86]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index=[0, 2, 4])

In [87]:
obj3

0      blue
2    purple
4    yellow
dtype: object

In [88]:
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

在DataFrame中,reindex可以改变行索引、列索引,也可以同时改变二者。  
当仅传入一个序列时,结果中的行会重建索引:

In [89]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=['a', 'c', 'd'],
                     columns=['Ohio', 'Texas', 'California'])

In [90]:
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [91]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])

In [92]:
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


列可以使用columns关键字重建索引:

In [93]:
states = ['Texas', 'Utah', 'California']

In [94]:
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


我们更深入地探索时,你可以使用loc进行更为简洁的标签索引,许多用户更倾向于使用这种方式:

In [95]:
test = ['a', 'b', 'c', 'd']
frame.loc[test, states]

/home/y/anaconda2/envs/notebook/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


,Texas,Utah,California
a,1.0,NaN,2.0
b,NaN,NaN,NaN
c,4.0,NaN,5.0
d,7.0,NaN,8.0


## 5.2.2 轴向上删除条目

如果你已经拥有索引数组或不含条目的列表,在轴向上删除一个或更多的条目就非常容易,但这样需要一些数据操作和集合逻辑,drop方法会返回一个含有指示值或轴向上删除值的新对象:

In [281]:
obj = pd.Series(np.arange(5), index=['a', 'b', 'c', 'd', 'e'])

In [282]:
obj

a    0
b    1
c    2
d    3
e    4
dtype: int64

In [283]:
new_obj = obj.drop('c') # 返回一个被删除后的新对象

In [284]:
new_obj

a    0
b    1
d    3
e    4
dtype: int64

In [285]:
obj.drop(['d', 'c'])

a    0
b    1
e    4
dtype: int64

在DataFrame中,索引值可以从轴向上删除。为了表明这个特性,我们首先创建一个示例DataFrame:

In [286]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [287]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


在调用drop时使用标签序列会根据行标签删除值(轴0):

In [288]:
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [289]:
data.drop(['Colorado', 'Ohio'], axis=0)

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [290]:
data.drop(['Colorado', 'Ohio'], axis='index')

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


你可以通过传递axis=1或axis='columns'来从列中删除值:

In [291]:
data.drop('two', axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [292]:
data.drop('two', axis='columns')

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


很多函数,例如drop,会修改Series或DataFrame的尺寸或形状,这些方法直接操作原对象而不返回新对象:

In [293]:
obj.drop('c', inplace=True) # 设置关键字参数inplace=True那么drop会直接在原数据对象上进行删除，也不会返回新对象

In [294]:
obj

a    0
b    1
d    3
e    4
dtype: int64

### <font color="#B22222">重点1</font>
<font color="#CD2626">
    <strong>
        请注意inplace属性,它会清除被删除的数据。
    </strong>
</font>

## 5.2.3 索引、选择与过滤

Series的索引(obj\[...\])与NumPy数组索引的功能类似,只不过Series的索引值可以不仅仅是整数。相关示例如下:

In [110]:
obj = pd.Series(np.arange(4.), index=['a', 'b', 'c', 'd'])

In [111]:
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [112]:
obj['b']

1.0

In [113]:
obj[1]

1.0

In [114]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [115]:
obj[['b', 'a', 'd']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [116]:
obj[[1, 3]]

b    1.0
d    3.0
dtype: float64

In [117]:
obj[obj<2]

a    0.0
b    1.0
dtype: float64

普通的Python切片中是不包含尾部的,Series的切片与之不同:

In [118]:
obj['b':'c']

b    1.0
c    2.0
dtype: float64

使用这些方法设值时会修改Series相应的部分:

In [119]:
obj['b':'c'] = 5

In [120]:
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

使用单个值或序列,可以从DataFrame中索引出一个或多个列:

In [121]:
data = pd.DataFrame(np.arange(16).reshape((4, 4,)),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])

In [122]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [123]:
data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int64

In [124]:
data[['three', 'one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


这种索引方式也有特殊案例。首先,可以根据一个布尔值数组切片或选择数据:

In [125]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [126]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


行选择语法data\[:2\]非常方便。传递单个元素或一个列表到\[\]符号中可以选择列。

另一个用例是使用布尔值DataFrame进行索引,布尔值DataFrame可以是对标量值进行比较产生的:

In [127]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [128]:
data[data < 5] = 0

In [129]:
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


### 5.2.3.1 使用loc和iloc选择数据

针对DataFrame在行上的标签索引,我将介绍特殊的索引符号loc和iloc。  
他们允许你使用轴标签(loc)或整数标签(iloc)以NumPy风格的语法从DataFrame中选出数组的行和列的子集。

我们通过标签选出单行多列的数据作为基础示例:

In [130]:
data.loc['Colorado', ['two', 'three']]

two      5
three    6
Name: Colorado, dtype: int64

然后我们使用整数标签iloc进行类似的数据选择:

In [131]:
data.iloc[2, [3, 0, 1]]

four    11
one      8
two      9
Name: Utah, dtype: int64

In [132]:
data.iloc[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int64

In [133]:
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
Colorado,7,0,5
Utah,11,8,9


除了单个标签或标签列表之外,索引功能还可以用于切片:

In [134]:
data.loc[:'Utah', 'two']

Ohio        0
Colorado    5
Utah        9
Name: two, dtype: int64

In [135]:
data.iloc[:, :3][data.three > 5]

,one,two,three
Colorado,0,5,6
Utah,8,9,10
New York,12,13,14


因此,有多种方式可以选择、重排pandas对象中的数据。表5-4提供了DataFrame选择数据的一些方法。之后的章节会介绍处理分层索引的可选参数。

## 5.2.4 整数索引

在pandas对象使上用整数索引对新用户来说经常会产生歧义,这是因为它和在列表、元组等Python内建数据结构上进行索引有些许不同。  
例如,你可能认为下面的代码会产生错误:

In [136]:
ser = pd.Series(np.arange(3.))

In [137]:
ser

0    0.0
1    1.0
2    2.0
dtype: float64

In [138]:
import traceback

try:
    ser[-1]
except Exception:
    traceback.print_exc()

# 当索引也是整数的时候，使用整数索引会找到index索引，会出错，所以个人觉得对pandas进行索引时最好使用loc和iloc方法

Traceback (most recent call last):
  File "<ipython-input-138-d58931709636>", line 4, in <module>
    ser[-1]
  File "/home/y/anaconda2/envs/notebook/lib/python3.6/site-packages/pandas/core/series.py", line 767, in __getitem__
    result = self.index.get_value(self, key)
  File "/home/y/anaconda2/envs/notebook/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 3118, in get_value
    tz=getattr(series.dtype, 'tz', None))
  File "pandas/_libs/index.pyx", line 106, in pandas._libs.index.IndexEngine.get_value
  File "pandas/_libs/index.pyx", line 114, in pandas._libs.index.IndexEngine.get_value
  File "pandas/_libs/index.pyx", line 162, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 958, in pandas._libs.hashtable.Int64HashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 964, in pandas._libs.hashtable.Int64HashTable.get_item
KeyError: -1


对于非整数索引,则不会有潜在的歧义:

In [139]:
ser2 = pd.Series(np.arange(3.), index=list('abc'))

In [140]:
ser2

a    0.0
b    1.0
c    2.0
dtype: float64

In [141]:
ser2[-1]

2.0

为了保持一致性,如果你有一个包含整数的轴索引,数据选择时请始终使用标签索引。  
为了更精确地处理,可以使用loc(用于标签)或iloc(用于整数):

In [142]:
ser[:1]

0    0.0
dtype: float64

In [143]:
ser.iloc[-1]

2.0

In [144]:
ser.loc[:1]

0    0.0
1    1.0
dtype: float64

In [145]:
ser.iloc[:1]

0    0.0
dtype: float64

### <font color="#B22222">重点1</font>

<font color="#949494">
    <strong>
        <font color="#9A32CD">
            整数索引
        </font>
        和numpy之类的索引是一样的，
        <font color="#9A32CD">
            包含头部不包含尾部
        </font>
        。
        <font color="#CD0000">
            而标签索引是头尾都包含。
        </font>
    </strong>
</font>

## 5.2.5 算术和数据对齐

不同索引的对象之间的算术行为是pandas提供给一些应用的一项重要特性。  
当你将对象相加时,如果存在某个索引对不相同,则返回结果的索引将是索引对的并集。对数据库用户来说,这个特性类似于索引标签的自动外连接(outer join)。  
让我们看下面的示例:

In [146]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=list('acde'))

In [147]:
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1], index=list('acefg'))

In [148]:
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [149]:
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [150]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

没有交叠的标签位置上,内部数据对齐会产生缺失值。  
缺失值会在后续的算术操作上产生影响。  
在DataFrame的示例中,行和列上都会执行对齐:

In [151]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), columns=list('bcd'),
                    index=['Ohio', 'Texas', 'Colorado'])

In [152]:
df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)), columns=list('bde'),
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])

In [153]:
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [154]:
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


将这些对象加在一起,返回一个DataFrame,它的索引、列是每个DataFrame的索引、列的并集:

In [155]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


由于'c'列和'e'列并不是两个DataFrame共有的列,这两列中产生了缺失值。  
对于行标签不同的DataFrame对象也是如此。

如果你将两个行或列完全不同的DataFrame对象相加,结果将全部为空:

In [156]:
df1 = pd.DataFrame({'A': [1, 2]})

In [157]:
df2 = pd.DataFrame({'B': [3, 4]})

In [158]:
df1

,A
0,1
1,2


In [159]:
df2

,B
0,3
1,4


In [160]:
df1 + df2

,A,B
0,NaN,NaN
1,NaN,NaN


### 5.2.5.1 使用填充值的算术方法

在两个不同的索引化对象之间进行算术操作时,你可能会想要使用特殊填充值,比如当轴标签在一个对象中存在,在另一个对象中不存在时,你想将缺失值填充为0:

In [161]:
df1 = pd.DataFrame(np.arange(12.).reshape((3, 4)),
                   columns=list('abcd'))

In [162]:
df2 = pd.DataFrame(np.arange(20.).reshape((4, 5)),
                   columns=list('abcde'))

In [163]:
df2.loc[1, 'b'] = np.nan

In [164]:
df1

,a,b,c,d
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0


In [165]:
df2

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,4.0
1,5.0,NaN,7.0,8.0,9.0
2,10.0,11.0,12.0,13.0,14.0
3,15.0,16.0,17.0,18.0,19.0


将这些df添加到一起会导致在一些不重叠的位置出现NA值:

In [166]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


在df1上使用add方法,我将df2和一个fill_value作为参数传入:

In [167]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


Series和DataFrame的算术方法中的每一个都有一个以r开头的副本,这些副本方法的参数是翻转的。  
因此下面两个语句的结果是等价的:

In [168]:
1 / df1

,a,b,c,d
0,inf,1.000000,0.500000,0.333333
1,0.250000,0.200000,0.166667,0.142857
2,0.125000,0.111111,0.100000,0.090909


In [169]:
df1.rdiv(1)

,a,b,c,d
0,inf,1.000000,0.500000,0.333333
1,0.250000,0.200000,0.166667,0.142857
2,0.125000,0.111111,0.100000,0.090909


与此相关的一点,当对Series或DataFrame重建索引时,你也可以指定一个不同的填充值:

In [170]:
df1.reindex(columns=df2.columns, fill_value=0)

,a,b,c,d,e
0,0.0,1.0,2.0,3.0,0
1,4.0,5.0,6.0,7.0,0
2,8.0,9.0,10.0,11.0,0


### <font color="#B22222">重点1:灵活算术方法</font>
<img src="第5章  pandas入门 数据/表5-5:灵活算术方法.png">

### 5.2.5.2 DataFrame和Series间的操作

DataFrame和Series间的算术操作与NumPy中不同维度数组间的操作类似。  
首先,在下面的生动示例中,考虑二维数组和其中一行之间的区别:

In [171]:
arr = np.arange(12.).reshape((3, 4))

In [172]:
arr

array([[ 0.,  1.,  2.,  3.],
       [ 4.,  5.,  6.,  7.],
       [ 8.,  9., 10., 11.]])

In [173]:
arr[0]

array([0., 1., 2., 3.])

In [174]:
arr - arr[0]

array([[0., 0., 0., 0.],
       [4., 4., 4., 4.],
       [8., 8., 8., 8.]])

当我们从arr中减去arr[0]时,减法在每一行都进行了操作。  
这就是所谓的广播机制,后续将会在附录A中涉及对通用NumPy数组的内容更深入的解释。  
DataFrame和Series间的操作是类似的:

In [175]:
frame = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                     columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])

In [176]:
series = frame.iloc[0]

In [177]:
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [178]:
series

b    0.0
d    1.0
e    2.0
Name: Utah, dtype: float64

默认情况下,DataFrame和Series的数学操作中会将Series的索引和DataFrame的列进行匹配,并广播到各行:

In [179]:
frame - series

,b,d,e
Utah,0.0,0.0,0.0
Ohio,3.0,3.0,3.0
Texas,6.0,6.0,6.0
Oregon,9.0,9.0,9.0


如果一个索引值不在DataFrame的列中,也不在Series的索引中,则对象会重建索引并形成联合:

In [180]:
series2 = pd.Series(range(3), index=list('bef'))
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


如果你想改为在列上进行广播,在行上匹配,你必须使用算术方法中的一种。例如:

In [181]:
series3 = frame.loc[:, 'd']

In [182]:
frame

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


In [183]:
series3

Utah       1.0
Ohio       4.0
Texas      7.0
Oregon    10.0
Name: d, dtype: float64

In [184]:
frame.sub(series3, axis=0)

,b,d,e
Utah,-1.0,0.0,1.0
Ohio,-1.0,0.0,1.0
Texas,-1.0,0.0,1.0
Oregon,-1.0,0.0,1.0


你传递的axis值是用于匹配轴的。上面的示例中表示我们需要在DataFrame的行索引上对行匹配(axis='index'或axis=0),并进行广播。

## 5.2.6 函数应用和映射

NumPy的通用函数(逐元素数组方法)对pandas对象也有效:

In [185]:
frame = pd.DataFrame(np.random.randn(4, 3), columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])

In [186]:
frame

,b,d,e
Utah,-2.055701,0.377657,-2.620879
Ohio,-0.078265,-1.505287,-1.011453
Texas,-0.430144,-0.452101,0.471442
Oregon,0.595729,0.235240,-1.878586


In [187]:
np.abs(frame)

,b,d,e
Utah,2.055701,0.377657,2.620879
Ohio,0.078265,1.505287,1.011453
Texas,0.430144,0.452101,0.471442
Oregon,0.595729,0.235240,1.878586


另一个常用的操作是将函数应用到一行或一列的一维数组上。DataFrame的apply方法可以实现这个功能:

In [188]:
f = lambda x: x.max() - x.min()

In [189]:
frame.apply(f)

b    2.651431
d    1.882944
e    3.092321
dtype: float64

这里的函数f,可以计算Series最大值和最小值的差,会被frame中的每一列调用一次。  
结果是一个以frame的列作为索引的Series。

如果你传递axis='columns'给apply函数,函数将会被每行调用一次:

In [190]:
frame.apply(f, axis=1)

Utah      2.998536
Ohio      1.427022
Texas     0.923543
Oregon    2.474315
dtype: float64

大部分最常用的数组统计(比如sum和mean)都是DataFrame的方法,因此计算统计值时使用apply并不是必需的。

传递给apply的函数并不一定要返回一个标量值,也可以返回带有多个值的Series:

In [191]:
def f(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])

In [192]:
frame.apply(f)

,b,d,e
min,-2.055701,-1.505287,-2.620879
max,0.595729,0.377657,0.471442


逐元素的Python函数也可以使用。假设你想要根据frame中的每个浮点数计算一个格式化字符串,可以使用applymap方法:

In [193]:
format = lambda x: '%.2f' % x

In [194]:
frame.applymap(format)

,b,d,e
Utah,-2.06,0.38,-2.62
Ohio,-0.08,-1.51,-1.01
Texas,-0.43,-0.45,0.47
Oregon,0.60,0.24,-1.88


使用applymap作为函数名是因为Series有map方法,可以将一个逐元素的函数应用到Series上:

In [195]:
frame.iloc[:, -1].map(format)

Utah      -2.62
Ohio      -1.01
Texas      0.47
Oregon    -1.88
Name: e, dtype: object

## 5.2.7 排序和排名

根据某些准则对数据集进行排序是另一个重要的内建操作。  
如需按行或列索引进行字典型排序,需要使用sort_index方法,该方法返回一个新的、排序好的对象:

In [196]:
obj = pd.Series(range(4), index=list('dabc'))

In [197]:
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

在DataFrame中,你可以在各个轴上按索引排序:

In [198]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)), 
                     index=['three', 'one'],
                     columns=list('dabc'))

In [199]:
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [200]:
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [201]:
frame.sort_index(axis=1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


数据默认会升序排序,但是也可以按照降序排序:

In [202]:
frame.sort_index(axis='columns', ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


如果要根据Series的值进行排序,使用sort_values方法:

In [203]:
obj = pd.Series([4, 7, -3, 2])

In [204]:
obj.sort_values()

2   -3
3    2
0    4
1    7
dtype: int64

默认情况下,所有的缺失值都会被排序至Series的尾部:

In [205]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])

In [206]:
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

当对DataFrame排序时,你可以使用一列或多列作为排序键。  
为了实现这个功能,传递一个或多个列名给sort_values的可选参数by:

In [207]:
frame = pd.DataFrame({'b': [4, 7, -3, 2], 'a': [0, 1, 0, 1]})

In [208]:
frame

,b,a
0,4,0
1,7,1
2,-3,0
3,2,1


In [209]:
frame.sort_values(by='b')

,b,a
2,-3,0
3,2,1
0,4,0
1,7,1


对多列排序时,传递列名的列表:

In [210]:
frame.sort_values(by=['a', 'b'])

,b,a
2,-3,0
0,4,0
3,2,1
1,7,1


排名是指对数组从1到有效数据点总数分配名次的操作。  
Series和DataFrame的rank方法是实现排名的方法,默认情况下,rank通过将平均排名分配到每个组来打破平级关系:

In [211]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4])

In [212]:
obj.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

排名也可以根据他们在数据中的观察顺序进行分配:

In [213]:
obj.rank(method='first')

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

在上面的例子中,对条目0和2设置的名次为6和7,而不是之前的平均排名6.5,是因为在数据中标签0在标签2的前面。

你可以按降序排名:

In [214]:
obj.rank(ascending=False, method='max')

0    2.0
1    7.0
2    2.0
3    4.0
4    5.0
5    6.0
6    4.0
dtype: float64

DataFrame可以对行或列计算排名:

In [215]:
frame = pd.DataFrame({'b': [4.3, 7, -3, 2], 'a': [0, 1, 0, 1], 'c': [-2, 5, 8, -2.5]})

In [216]:
frame

,b,a,c
0,4.3,0,-2.0
1,7.0,1,5.0
2,-3.0,0,8.0
3,2.0,1,-2.5


In [217]:
frame.rank(axis=1)

,b,a,c
0,3.0,2.0,1.0
1,3.0,1.0,2.0
2,1.0,2.0,3.0
3,3.0,2.0,1.0


### <font color="#B22222">重点1:排名中的平级关系打破方法</font>
<img src="第5章  pandas入门 数据/表5-6:排名中的平级关系打破方法.png">

## 5.2.8 含有重复标签的轴索引

目前为止我们所见过的示例中,轴索引都是唯一的(索引值)。  
尽管很多pandas函数(比如reindex)需要标签是唯一的,但这个并不是强制性的。  
让我们考虑一个小型的带有重复索引的Series:

In [218]:
obj = pd.Series(range(5), index=list('aabbc'))

In [219]:
obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

索引的is_unique属性可以告诉你它的标签是否唯一:

In [220]:
obj.index.is_unique

False

带有重复索引的情况下,数据选择是与之前操作有差别的主要情况。  
根据一个标签索引多个条目会返回一个序列,而单个条目会返回标量值:

In [221]:
obj.loc['a']

a    0
a    1
dtype: int64

In [222]:
obj.loc['c']

4

这可能会使代码更复杂,因为来自索引的输出类型可能因标签是否重复而有所不同。

相同的逻辑可以拓展到在DataFrame中进行行索引:

In [223]:
df = pd.DataFrame(np.random.randn(4, 3), index=list('aabb'))

In [224]:
df

,0,1,2
a,-0.666637,-0.002350,1.052409
a,-0.166141,0.632073,-1.823114
b,-1.031050,0.814217,-0.783293
b,0.483019,-1.039297,-0.066216


In [225]:
df.loc['a']

,0,1,2
a,-0.666637,-0.002350,1.052409
a,-0.166141,0.632073,-1.823114


# 5.3 描述性统计的概述与计算

pandas对象装配了一个常用数学、统计学方法的集合。  
其中大部分属于归约或汇总统计的类别,这些方法从DataFrame的行或列中抽取一个Series或一系列值的单个值(如总和或平均值)。  
与NumPy数组中的类似方法相比,它们内建了处理缺失值的功能。考虑一个小型DataFrame:

In [226]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5], [np.nan, np.nan], [0.75, -1.3]],
                  index=list('abcd'), columns=['one', 'two'])

In [227]:
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


调用DataFrame的sum方法返回一个包含列上加和的Series:

In [228]:
df.sum()

one    9.25
two   -5.80
dtype: float64

传入axis='columns'或axis=1,则会将一行上各个列的值相加:

In [229]:
df.sum(axis=1)

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

除非整个切片上(在本例中是行或列)都是NA,否则NA值是被自动排除的。可以通过禁用skipna来实现不排除NA值:

In [230]:
df.mean(axis=1, skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

### <font color="#B22222">重点1:归约方法可选参数</font>
<img src="第5章  pandas入门 数据/表5-7:归约方法可选参数.png">

一些方法,比如idxmin和idxmax,返回的是间接统计信息,比如最小值或最大值的索引值:

In [231]:
df.idxmin()

one    d
two    b
dtype: object

In [232]:
df.idxmax()

one    b
two    d
dtype: object

除了归约方法外,有的方法是积累型方法:

In [233]:
df.cumsum() # 从列方向，第二个有效元素的值是第一个元素和第二个元素之和

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


还有一类方法既不是归约型方法也不是积累型方法。describe就是其中之一,它一次性产生多个汇总统计:

In [234]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


对于非数值型数据,describe产生另一种汇总统计:

In [235]:
obj = pd.Series(list('aabc') * 4)

In [236]:
obj

0     a
1     a
2     b
3     c
4     a
5     a
6     b
7     c
8     a
9     a
10    b
11    c
12    a
13    a
14    b
15    c
dtype: object

In [237]:
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

### <font color="#B22222">重点2:描述性统计和汇总统计</font>
<img src="第5章  pandas入门 数据/表5-8:描述性统计和汇总统计.png">

### 5.3.1 相关性和协方差

为了获得一些股票行情,我使用pandas_datareader模块下载一些数据:

In [238]:
import pandas_datareader.data as web


all_data = {ticker: web.get_data_yahoo(ticker) for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']}
price = pd.DataFrame({ticker: data['Adj Close'] for ticker, data in all_data.items()})
volume = pd.DataFrame({ticker: data['Volume'] for ticker, data in all_data.items()})

现在我计算股价的百分比,还有一些时间序列操作将会在第11章中深入探索:

In [239]:
returns = price.pct_change() # 变化率计算方式是后一个元素和前一个元素的差除以前一个元素

In [240]:
returns.tail()

,AAPL,IBM,MSFT,GOOG
Date,,,,
2019-03-11,0.034642,0.019395,0.020994,0.029274
2019-03-12,0.011235,0.004139,0.007002,0.014833
2019-03-13,0.004422,0.002025,0.007745,0.000101
2019-03-14,0.011117,0.001660,0.000786,-0.006511
2019-03-15,0.013008,0.004611,0.011519,-0.000919


Series的corr方法计算的是两个Series中重叠的、非NA的、按索引对齐的值的相关性。相应地,cov计算的是协方差:

In [241]:
returns['MSFT'].corr(returns['IBM'])

0.48744198209142764

In [242]:
returns['MSFT'].cov(returns['IBM'])

8.725998369002742e-05

由于MSFT是一个有效的Python属性,我们可以使用更为简洁的语法来获得这些数据:

In [243]:
returns.MSFT.corr(returns.IBM)

0.48744198209142764

另一方面,DataFrame的corr和cov方法会分别以DataFrame的形式返回相关性和协方差矩阵:

In [244]:
returns.corr()

,AAPL,IBM,MSFT,GOOG
AAPL,1.000000,0.373229,0.451176,0.458288
IBM,0.373229,1.000000,0.487442,0.408997
MSFT,0.451176,0.487442,1.000000,0.537773
GOOG,0.458288,0.408997,0.537773,1.000000


In [245]:
returns.cov()

,AAPL,IBM,MSFT,GOOG
AAPL,0.000270,0.000076,0.000108,0.000116
IBM,0.000076,0.000152,0.000087,0.000078
MSFT,0.000108,0.000087,0.000210,0.000120
GOOG,0.000116,0.000078,0.000120,0.000238


使用DataFrame的corrwith方法,你可以计算出DataFrame中的行或列与另一个序列或DataFrame的相关性。  
该方法传入一个Series时,会返回一个含有为每列计算相关性值的Series:

In [246]:
returns.corrwith(returns.loc[:, 'IBM'])

AAPL    0.373229
IBM     1.000000
MSFT    0.487442
GOOG    0.408997
dtype: float64

传入一个DataFrame时,会计算匹配到列名的相关性数值。  
在这里,我计算出交易量百分比变化的相关性:

In [247]:
returns.corrwith(volume)

AAPL   -0.060553
IBM    -0.152792
MSFT   -0.089384
GOOG   -0.017752
dtype: float64

传入axis='columns'会逐行地进行计算。  
在所有例子中,在计算相关性之前,数据点已经按标签进行了对齐。

In [296]:
returns.corrwith(returns.iloc[1], axis=1)

Date
2009-12-31         NaN
2010-01-04    1.000000
2010-01-05    0.771924
2010-01-06    0.425157
2010-01-07    0.562030
2010-01-08   -0.959377
2010-01-11   -0.671724
2010-01-12   -0.069910
2010-01-13    0.982733
2010-01-14   -0.115973
2010-01-15    0.118079
2010-01-19    0.410638
2010-01-20    0.222989
2010-01-21   -0.913025
2010-01-22    0.110035
2010-01-25    0.882193
2010-01-26    0.705987
2010-01-27    0.858003
2010-01-28   -0.602590
2010-01-29   -0.995748
2010-02-01    0.034360
2010-02-02    0.403934
2010-02-03    0.013956
2010-02-04   -0.723244
2010-02-05    0.521142
2010-02-08   -0.667956
2010-02-09    0.650711
2010-02-10    0.105629
2010-02-11    0.557223
2010-02-12    0.318550
                ...   
2019-02-01   -0.270106
2019-02-04    0.781102
2019-02-05    0.683633
2019-02-06    0.358388
2019-02-07    0.033986
2019-02-08    0.498431
2019-02-11   -0.903344
2019-02-12   -0.785367
2019-02-13   -0.523485
2019-02-14    0.511262
2019-02-15    0.278726
2019-02-19   -0.770689
2019-0

## 5.3.2 唯一值、计数和成员属性

另一类相关的方法可以从一维Series包含的数值中提取信息。  
为了说明这些方法,请考虑这个例子:

In [249]:
obj = pd.Series(list('cadaabbcc'))

第一个函数是unique,它会给出Series中的唯一值:

In [250]:
uniques = obj.unique()

In [251]:
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

唯一值并不一定按照排序好的顺序返回,但是如果需要的话可以进行排序(uniques.sort())。  
相应地,value_counts计算Series包含的值的个数:

In [252]:
obj.value_counts()

c    3
a    3
b    2
d    1
dtype: int64

为了方便,返回的Series会按照数量降序排序。  
value_counts也是有效的pandas顶层方法,可以用于任意数组或序列:

In [253]:
pd.value_counts(obj.values, sort=False)

a    3
d    1
b    2
c    3
dtype: int64

isin执行向量化的成员属性检查,还可以将数据集以Series或DataFrame一列的形式过滤为数据集的值子集:

In [254]:
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

In [255]:
mask = obj.isin(['b', 'c'])

In [256]:
mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [257]:
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

与isin相关的Index.get_indexer方法,可以提供一个索引数组,这个索引数组可以将可能非唯一值数组转换为另一个唯一值数组:

In [258]:
to_match = pd.Series(list('cabbca'))

In [259]:
unique_vals = pd.Series(list('cba'))

In [260]:
pd.Index(unique_vals).get_indexer(to_match)
# 表示，to_match 中的字符，在 unoque_vals 中的位置索引

array([0, 2, 1, 1, 0, 2])

### <font color="#B22222">重点1:唯一值、计数和集合成员属性方法</font>
<img src="第5章  pandas入门 数据/表5-9:唯一值、计数和集合成员属性方法.png">

某些情况下,你可能想要计算DataFrame多个相关列的直方图,如下面的例子:

In [297]:
data = pd.DataFrame({'Ou1': [1, 3, 4, 3, 4],
                     'Ou2': [2, 3, 1, 2, 3],
                     'Ou3': [1, 5, 2, 4, 4]})

In [298]:
data

,Ou1,Ou2,Ou3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


将pandas.value_counts传入DataFrame的apply函数可以得到:

In [299]:
result = data.apply(pd.value_counts).fillna(0)

In [301]:
result # 结果的index是原dataframe的值，结果的value是原dataframe的值在原dataframe每列出现的次数统计

,Ou1,Ou2,Ou3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0


这里,结果中的行标签是所有列中出现的不同值,数值则是这些不同值在每个列中出现的次数。